<a href="https://colab.research.google.com/github/sebabecerra/Fondos-de-Pensiones-Codes/blob/main/Rentabilidad_FP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Comparacion de Fondos de Pensiones a Diciembre 2025

In [147]:
import pandas as pd
import plotly.graph_objects as go

# ----------------------------------------
# 1. Leer archivo Excel
# ----------------------------------------
file_path = "/content/drive/MyDrive/FondosdePensiones/rentabilidad_real_mensual_fondos_deflactada_uf.xls"
df = pd.read_excel(file_path, skiprows=3, engine="xlrd")

# ----------------------------------------
# 2. Renombrar columnas
# ----------------------------------------
df.columns = ['Fecha_raw','A','B','C','D','E']

# ----------------------------------------
# 3. Convertir fechas y eliminar inválidas
# ----------------------------------------
df['Fecha_dt'] = pd.to_datetime(df['Fecha_raw'], errors='coerce')
df = df.dropna(subset=['Fecha_dt'])

# ----------------------------------------
# 4. Filtrar período 2021–2025
# ----------------------------------------
df = df[(df['Fecha_dt'] >= '2021-01-01') & (df['Fecha_dt'] <= '2025-12-31')]

# ----------------------------------------
# 5. Definir fondos y nombres
# ----------------------------------------
fondos = ['A','B','C','D','E']
nombres = {
    'A':'Fondo A Más riesgoso',
    'B':'Fondo B Riesgoso',
    'C':'Fondo C Intermedio',
    'D':'Fondo D Conservador',
    'E':'Fondo E Más conservador'
}

# ----------------------------------------
# 6. Pasar % a decimal
# ----------------------------------------
df[fondos] = df[fondos] / 100

# ----------------------------------------
# 7. Calcular rentabilidad acumulada
# ----------------------------------------
df_acum = df.copy()
for f in fondos:
    df_acum[f] = (1 + df_acum[f]).cumprod() - 1   # acumulado en decimal
    df_acum[f] = (df_acum[f] * 100).round(2)      # pasar a %

# ----------------------------------------
# 8. Definir ticks en junio mostrando solo año
# ----------------------------------------
ticks_x = df_acum[df_acum['Fecha_dt'].dt.month == 6]['Fecha_dt']
ticks_text = ticks_x.dt.year.astype(str)

# ----------------------------------------
# 9. Crear figura
# ----------------------------------------
fig = go.Figure()

# Agregar una línea por cada fondo
for f in fondos:
    fig.add_trace(go.Scatter(
        x=df_acum['Fecha_dt'],
        y=df_acum[f],
        mode='lines',
        name=nombres[f],
        hovertemplate=f"{nombres[f]}<br>Fecha: %{{x|%b-%Y}}<br>Rentabilidad: %{{y:.2f}}%<extra></extra>"
    ))

# Línea horizontal en 0%
fig.add_hline(y=0)

# ----------------------------------------
# 10. Formato del gráfico
# ----------------------------------------
fig.update_layout(
    height=800,
    width=1200,
    title=dict(
        text="Rentabilidad Real Acumulada de los Fondos de Pensiones<br>(Periodo enero 2021 - diciembre 2025)",
        x=0.5,
        xanchor="center",
        font=dict(size=22)
    ),
    yaxis=dict(
        range=[-25,25],
        autorange=False,
        tickmode="array",
        tickvals=[-25,-20,-15,-10,-5,0,5,10,15,20,25],
        ticksuffix="%",
        showgrid=True,
        zeroline=True,
        ticklabelstandoff=10,
        tickfont=dict(size=14)
    ),
    xaxis=dict(
        tickmode="array",
        tickvals=ticks_x,
        ticktext=ticks_text,
        ticklabelstandoff=30,
        automargin=True,
        tickfont=dict(size=14)
    ),
    template="plotly_white",
    legend=dict(orientation='h', y=-0.1, x=0.5, xanchor='center'),
    margin=dict(l=80, r=40, t=100, b=120)
)

# Mostrar gráfico
fig.show()


In [148]:
import pandas as pd
import plotly.graph_objects as go

# -------------------------------------------------
# 1. Leer archivo Excel
# -------------------------------------------------
file_path = "/content/drive/MyDrive/FondosdePensiones/rentabilidad_real_mensual_fondos_deflactada_uf.xls"
df = pd.read_excel(file_path, skiprows=3, engine="xlrd")

# -------------------------------------------------
# 2. Renombrar columnas
# -------------------------------------------------
df.columns = ['Fecha_raw','A','B','C','D','E']

# -------------------------------------------------
# 3. Convertir fechas y eliminar filas inválidas
# -------------------------------------------------
df['Fecha_dt'] = pd.to_datetime(df['Fecha_raw'], errors='coerce')
df = df.dropna(subset=['Fecha_dt'])

# -------------------------------------------------
# 4. Pasar rentabilidades de % a decimal
# -------------------------------------------------
fondos = ['A','B','C','D','E']
df[fondos] = df[fondos] / 100

# -------------------------------------------------
# 5. Filtrar período de análisis
# -------------------------------------------------
df = df[(df['Fecha_dt'] >= '2002-09-01') & (df['Fecha_dt'] <= '2025-12-31')]

# -------------------------------------------------
# 6. Calcular índice acumulado normal
# -------------------------------------------------
df_idx = df.copy()
for f in fondos:
    df_idx[f] = (1 + df_idx[f]).cumprod()

# -------------------------------------------------
# 7. Forzar septiembre 2002 = 1 para todos los fondos
# -------------------------------------------------
base_date = pd.to_datetime("2002-09-01")
mask_base = df_idx['Fecha_dt'].dt.to_period("M") == base_date.to_period("M")

# Forzar manualmente que todos valgan 1 en ese mes
for f in fondos:
    df_idx.loc[mask_base, f] = 1

# Reescalar toda la serie usando ese punto como base
base_vals = df_idx.loc[mask_base, fondos].iloc[0]
for f in fondos:
    df_idx[f] = df_idx[f] / base_vals[f]

# -------------------------------------------------
# 8. Definir ticks: cada 2 años (enero de años impares)
# -------------------------------------------------
ticks_x = df_idx[
    (df_idx['Fecha_dt'].dt.month == 6) &
    (df_idx['Fecha_dt'].dt.year % 2 == 1)
]['Fecha_dt']
ticks_text = ticks_x.dt.year.astype(str)

# -------------------------------------------------
# 9. Crear gráfico Plotly
# -------------------------------------------------
fig = go.Figure()

nombres = {
    'A':'Fondo A',
    'B':'Fondo B',
    'C':'Fondo C',
    'D':'Fondo D',
    'E':'Fondo E'
}

# Agregar líneas usando loop (misma estructura para todos)
for f in fondos:
    fig.add_trace(go.Scatter(
        x=df_idx['Fecha_dt'],
        y=df_idx[f],
        mode='lines',
        name=nombres[f],
        hovertemplate=f"{nombres[f]}<br>Fecha: %{{x|%b-%Y}}<br>Índice: %{{y:.2f}}<extra></extra>"
    ))

# -------------------------------------------------
# 10. Formato del gráfico
# -------------------------------------------------
fig.update_layout(
    height=800,
    width=1200,
    title=dict(
        text="Rentabilidad Histórica Real Acumulada (Índice base Sep-2002 = 1)<br>Período septiembre 2002 - diciembre 2025",
        x=0.5,
        xanchor="center",
        font=dict(size=22)
    ),
    yaxis=dict(
        range=[0, 4],
        autorange=False,
        showgrid=True,
        zeroline=False,
        ticklabelstandoff=10,
        tickfont=dict(size=14),
        tickformat=".2f"      # dos decimales en eje Y
    ),
    xaxis=dict(
        tickmode="array",
        tickvals=ticks_x,
        ticktext=ticks_text,
        ticklabelstandoff=30,
        automargin=True,
        tickfont=dict(size=14)
    ),
    template="plotly_white",
    legend=dict(
        orientation='h',
        y=-0.1,
        x=0.5,
        xanchor='center'
    ),
    margin=dict(l=80, r=40, t=100, b=120)
)

fig.show()


In [153]:
import numpy as np

resumen = []

for f in fondos:
    r = df[f]

    # Rentabilidad anual
    rent_total = (1+r).prod() - 1
    n_anios = len(r)/12
    rent_anual = (1+rent_total)**(1/n_anios) - 1

    # Volatilidad anual
    vol_anual = r.std() * np.sqrt(12)

    # Eficiencia retorno/riesgo
    eficiencia = rent_anual / vol_anual

    # Drawdown
    curva = (1+r).cumprod()
    max_prev = curva.cummax()
    drawdown = (curva/max_prev - 1).min()

    # Meses positivos
    pct_positivos = (r > 0).mean()

    resumen.append([
        f,
        rent_anual*100,
        vol_anual*100,
        eficiencia,
        drawdown*100,
        pct_positivos*100
    ])

tabla = pd.DataFrame(resumen, columns=[
    "Fondo",
    "Rentabilidad anual %",
    "Volatilidad anual %",
    "Eficiencia",
    "Max Drawdown %",
    "% Meses positivos"
]).round(2)

tabla


,Fondo,Rentabilidad anual %,Volatilidad anual %,Eficiencia,Max Drawdown %,% Meses positivos
0,A,3.75,9.69,0.39,-25.27,58.33
1,B,3.10,7.88,0.39,-19.58,56.67
2,C,1.14,7.02,0.16,-19.18,51.67
3,D,-0.02,8.60,-0.00,-20.78,51.67
4,E,0.06,9.76,0.01,-18.76,55.00
